In [ ]:
#sns.heatmap(traindf.corr(),annot=True,cmap='RdYlGn',linewidths=0.2) #data.corr()-->correlation matrix
#fig=plt.gcf()
#fig.set_size_inches(20,12)
#plt.show()

In [1]:
import numpy as np
import matplotlib.pyplot as  plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import seaborn as sns

In [2]:
path = 'F:/Books/Machine Learning/DataSets/Titanic'
train_data = pd.read_csv(path + '/train.csv')
test_data = pd.read_csv(path + '/test.csv')
sub_data = pd.read_csv(path + '/gender_submission.csv')

In [3]:
print(train_data.shape)
print(test_data.shape)
print(sub_data.shape)

(891, 12)
(418, 11)
(418, 2)


In [4]:
all_data  = pd.concat([train_data.drop(['Survived'],axis=1),test_data],axis=0)
all_data.shape

(1309, 11)

In [5]:
raw_label = train_data.Survived
raw_label.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [6]:
train_data.isna().sum()[train_data.isna().sum() != 0 ]

Age         177
Cabin       687
Embarked      2
dtype: int64

In [7]:
test_data.isna().sum()[test_data.isna().sum() != 0 ]

Age       86
Fare       1
Cabin    327
dtype: int64

In [8]:
all_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
drop_feat = ['PassengerId','Name']

In [18]:
all_data.Embarked.fillna(all_data.Embarked.mode().tolist()[0], inplace=True)
all_data.Fare.fillna(all_data.Fare.mode().tolist()[0], inplace=True)
m = all_data.Age.mean()
std = all_data.Age.std()
all_data.Age[all_data.Age.isna()] = np.random.normal(m,std,all_data.Age.isna().sum())

In [21]:
all_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
temp_data = all_data.copy()

In [ ]:
temp_data.Sex[temp_data.Sex == 'male'] = 1
temp_data.Sex[temp_data.Sex == 'female'] = 2


In [ ]:
temp_data.Embarked[temp_data.Embarked == 'S'] = 1
temp_data.Embarked[temp_data.Embarked == 'C'] = 2
temp_data.Embarked[temp_data.Embarked == 'Q'] = 3

In [ ]:
temp_data.head()

In [ ]:
temp_data.Cabin[temp_data.Cabin.isna()] = 1
temp_data.Cabin[temp_data.Cabin != False] = 2
temp_data.head()

In [ ]:
all_data.Embarked.fillna(all_data.Embarked.mode().tolist()[0], inplace=True)
all_data.Fare.fillna(all_data.Fare.mode().tolist()[0], inplace=True)
all_data.isna().sum()

In [ ]:
all_data.Cabin[all_data.Cabin.isna()] = 0
all_data.Cabin[all_data.Cabin != False] = 1
all_data.head()

In [ ]:
all_data.isna().sum()

In [ ]:
all_data['Sex'] = pd.get_dummies(all_data.Sex,drop_first=True,prefix='Sex')


In [ ]:
from sklearn.decomposition import NMF

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
all_data.Embarked = lb.fit_transform(all_data.Embarked)

In [ ]:
all_data.head()

In [ ]:
nmf = NMF(4)
feat = ['Pclass','Sex','Age','Fare','Cabin','Embarked']

In [ ]:
a.shape

In [ ]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])

                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        if e < 0.001:
            break
    return P, Q.T


In [ ]:
import numpy

In [ ]:
R = numpy.array(temp_data[feat].fillna(0))
N = len(R)
M = len(R[0])
K = 4 
P = numpy.random.rand(N,K)
Q = numpy.random.rand(M,K)
nP, nQ = matrix_factorization(R, P, Q, K)
nR = numpy.dot(nP, nQ.T)

In [ ]:
for i in range(5):
    for j in range(4):
        nR[i][j] = round(nR[i][j])

In [ ]:
non_negative_factorization

In [ ]:
a = nmf.fit_transform(temp_data[feat].fillna(0))

In [ ]:
temp_data = all_data.copy()

In [ ]:
temp_data.Age.fillna(0,inplace=True)

In [ ]:
feat = ['Pclass','Sex','Age','Fare','Cabin','Embarked']


In [ ]:
from sklearn.decomposition import non_negative_factorization

In [ ]:
from sklearn.preprocessing import Normalizer
pd.DataFrame(Normalizer().fit_transform(temp_data[feat].fillna(0)))

In [ ]:
a,b,c = non_negative_factorization(temp_data[feat].fillna(0) , n_components = 4)

In [ ]:
all_data['Sex'] = pd.get_dummies(all_data.Sex,drop_first=True)
all_data['CatAge'] = pd.qcut(all_data.Age,  q=4, labels=False)
all_data['CatFare']= pd.qcut(all_data.Fare, q=4,labels=False)
all_data.Cabin[all_data.Cabin.isna()] = 0
all_data.Cabin[all_data.Cabin != False] = 1
all_data.head()

In [ ]:
drop_feat = ['PassengerId','Name','Cabin','Ticket','Age','Fare']
all_data.drop(drop_feat,axis=1,inplace=True)

In [ ]:
all_data.head()

In [ ]:
all_data.Embarked.fillna(all_data.Embarked.mode().tolist()[0], inplace=True)
all_data.CatFare.fillna(all_data.CatFare.mode().tolist()[0], inplace=True)
all_data.isna().sum()

In [ ]:
all_data = pd.concat([all_data,pd.get_dummies(all_data.Pclass,prefix='Pclass',drop_first=True),
pd.get_dummies(all_data.Embarked,drop_first=True,prefix='Embarked'),
pd.get_dummies(all_data.CatAge,prefix='CatAge',drop_first=True),
pd.get_dummies(all_data.CatFare,prefix='CatFare',drop_first=True)],axis=1)
all_data.head()

In [ ]:
drop_feat = ['Pclass','Embarked','CatAge','CatFare']
all_data.drop(drop_feat,axis=1,inplace=True)

In [ ]:
all_data.head()

In [ ]:
from sklearn.preprocessing import LabelBinarizer 
lb = LabelBinarizer()
train_data.Sex = lb.fit_transform(train_data.Sex) # male = 1